In [1]:
import uproot
import awkward as ak
import numpy as np
import math
import hist
import matplotlib.pyplot as plt
import os
import subprocess
import vector
import gc
from itertools import product
import sys
import ROOT
from tqdm import tqdm

from Plot import Plot

vector.register_awkward() 
DATATYPE="mc"
BASEDIR="/pbs/throng/training/nantes-m2-rps-exp/data"
SAMPLE_RUN = 291694
MUON_MASS = 0.105
D_ABS = 500

events_tree = f"{BASEDIR}/run{SAMPLE_RUN}.{DATATYPE}.root:eventsTree"

if DATATYPE=="mc":
    gen_tree = f"{BASEDIR}/run{SAMPLE_RUN}.{DATATYPE}.root:genTree"

In [2]:
def getTracks(events):
    return ak.zip({ "px":events["Muon_Px"],
                    "py":events["Muon_Py"],
                    "pz":events["Muon_Pz"],
                    "E":events["Muon_E"],
                    "DCAx":events["Muon_xDCA"],
                    "DCAy":events["Muon_yDCA"],
                    "DCAz":events["Muon_zDCA"],
                    "charge":events["Muon_Charge"],
                    "thetaAbs":events["Muon_thetaAbs"],
                    "matched":events["Muon_matchedTrgThreshold"],
                    "PDGcode":events["PDGcode"]},
                    with_name='Momentum4D')

def getgenTracks(genevents):
    return ak.zip({ "px":genevents["Muon_GenPx"],
                    "py":genevents["Muon_GenPy"],
                    "pz":genevents["Muon_GenPz"],
                    "E":genevents["Muon_GenE"]},
                    with_name='Momentum4D')

In [3]:
def applySelection(data, criteria, plot=False, **kwargs):
    # Select data using function "criteria"
    selected = data[criteria(data)]
    if plot:
        hist_before = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=20,name='$|p|$'))
        hist_after = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=20,name='$|p|$'))
        try:
            hist_before.fill(ak.flatten(data.pt))
            hist_after.fill(ak.flatten(selected.pt))
        except AttributeError:
            hist_before.fill(ak.flatten(getTracks(data).pt))
            hist_after.fill(ak.flatten(getTracks(selected).pt))
        Plot(hist_before, hist_after, **kwargs)
    return selected

In [4]:
def get_pDCA_infos(goodTracks):
    DCA=np.sqrt(goodTracks.DCAx**2 + goodTracks.DCAy**2 + goodTracks.DCAz**2)
    p0=ak.where(goodTracks.thetaAbs < 3 , goodTracks.p - 3.0, goodTracks.p - 2.4)
    pDCA=p0*DCA
    
    N=3 #choice of resolution should be a number between 1 and 10
    sigmaAbs=ak.where(goodTracks.thetaAbs < 3, 80.0, 54.0) #factor used due to the crossed materials 
    pRes= 0.0004 * goodTracks.p
    sigmap= sigmaAbs/ (1 - ((N*pRes)/(1 + N*pRes)))

    thetaRes= 0.0005 * goodTracks.p
    sigmatheta= 535 * thetaRes

    #computing total resolution for pDCA
    sigmapDCA=np.sqrt(sigmap**2 + sigmatheta**2)

    goodTracks["pDCA"] = pDCA

    return pDCA, N, sigmapDCA

In [5]:
def analyzeGenTree():
    assert gen_tree
    for batch in uproot.iterate(gen_tree,
                                ["zVtxMC", "nMuonsGen", "Muon_GenPx","Muon_GenPy","Muon_GenPz","Muon_GenE","Muon_GenLabel","Muon_GenMotherPDGCode"],                                
                                 report=True):
        events = batch[0]

        codes = events.Muon_GenMotherPDGCode
        n_jpsi = sum(ak.flatten(codes) == 443)/2
        n_psi2s = sum(ak.flatten(codes) == 100443)/2

        return codes, n_jpsi, n_psi2s
        
        #Plot(hist_E)
        #Plot(hist_z)

In [6]:
def analyzeEventsTree(plot=False):
    for batch in uproot.iterate(events_tree,
                                ["zVtx", "nMuons", "isCINT","isCMUL","isCMSL","Muon_Px","Muon_Py","Muon_Pz","Muon_E","Muon_xDCA","Muon_yDCA","Muon_zDCA","Muon_Charge","Muon_thetaAbs","Muon_matchedTrgThreshold"],                                
                                 report=True):
        events=batch[0]

        PDGcodes, n_jpsi, n_psi2s = analyzeGenTree()
        events["PDGcode"] = PDGcodes[..., 0]

        def select_charge(events):
            charge = []
            for charges in events["Muon_Charge"]:
                if (-1 in charges) and (1 in charges):
                    charge += [True]
                else:
                    charge += [False]
            return charge

        def select_z(events):
            z = events["zVtx"]
            return (z < 10) & (z > -10)

        select_nb = lambda events: events["nMuons"] >= 2

        goodEvents = applySelection(events, select_nb, plot=plot, title="Nb selection")
        goodEvents = applySelection(goodEvents, select_z, plot=plot, title="z selection")
        goodEvents = applySelection(goodEvents, select_charge, plot=plot, title="charge selection")

        tracks = getTracks(goodEvents)

        select_pt = lambda tracks: tracks.pt > 1
        select_match = lambda tracks: tracks.matched >= 1
        select_eta = lambda tracks: (tracks.eta < -2.5) & (tracks.eta > -4)
        select_Rabs = lambda tracks: (D_ABS * np.tan(tracks.thetaAbs*np.pi/180) < 89.5) & (D_ABS * np.tan(tracks.thetaAbs*np.pi/180) > 17.5)
    
        
        goodTracks = applySelection(tracks, select_pt, plot=plot, title="pt selection")
        goodTracks = applySelection(goodTracks, select_match, plot=plot, title="match selection")
        goodTracks = applySelection(goodTracks, select_eta, plot=plot, title="eta selection")
        goodTracks = applySelection(goodTracks, select_Rabs, plot=plot, title="R_abs selection")

        pDCA, N, sigmapDCA = get_pDCA_infos(goodTracks)
        # for N in range(1, 15):
            # select_pDCA = lambda tracks: pDCA < N*sigmapDCA
            # goodTracks_ = applySelection(goodTracks, select_pDCA, plot=True, title=f"pDCA selection, N = {N}")
        select_pDCA = lambda tracks: pDCA < N*sigmapDCA
        goodTracks_ = applySelection(goodTracks, select_pDCA, plot=plot, title=f"pDCA selection, N = {N}")
        invariant_mass = []
        invariant_mass_psi2s = []
        # Pair tracks and calculate invarant mass
        for eventTracks in goodTracks:
            if len(eventTracks) < 2: continue
            positive = eventTracks[eventTracks.charge == 1] #Tracks with positive charge
            negative = eventTracks[eventTracks.charge == -1] #Tracks with negative charge
            matched_pair = [] # List to store tracks already paired
            for p, n in product(positive, negative):
                if p in matched_pair or n in matched_pair: continue
                E = p.E + n.E
                Pz = p.pz + n.pz
                y = 0.5*np.log((E + Pz) / (E - Pz))
                if y > -4 and y < -2.5:
                    if eventTracks.PDGcode[0] == 443:
                        invariant_mass.append(np.sqrt(2*MUON_MASS**2 + 2*p.dot(n)))
                    else:
                        invariant_mass_psi2s.append(np.sqrt(2*MUON_MASS**2 + 2*p.dot(n)))
                    matched_pair.append(p)
                    matched_pair.append(n)
                    
        if plot: 
            mass_hist = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=5,name='$|pt|$'))
            mass_hist.fill(invariant_mass)
            Plot(mass_hist, title="Invariant mass", save=True)
            mass_hist2 = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=5,name='$|pt|$'))
            mass_hist2.fill(invariant_mass_psi2s)
            Plot(mass_hist2, title="Invariant mass Psi2S", save=True)
            print(mass_hist.sum(), "J/psi")
            print(mass_hist2.sum(), "Psi(2S)")

        return invariant_mass, invariant_mass_psi2s, n_jpsi, n_psi2s
        

        # Plot(vhMagRaw, vhMagRaw2, labelx=r"zVtx", save=True, savename="zVtx_cut")


In [11]:
inv_mass_jpsi, inv_mass_psi2s = [], []
n_jpsi, n_psi2s = 0, 0
files = os.listdir(f"{BASEDIR}/")
def filter_file(file):
    if file == "merged_data.root": return False
    run, datatype, extension = file.split(".")
    return datatype == "mc"

filtered_files = list(filter(filter_file, files))
with tqdm(total = len(filtered_files), desc="Attends, ça calcul là... ", bar_format='{l_bar}{bar:30}{r_bar}{bar:-30b}') as bar:
    for file in filtered_files:
        # print(f"Analyzing file {file}")
        events_tree = f"{BASEDIR}/{file}:eventsTree"
        gen_tree = f"{BASEDIR}/{file}:genTree"
        data = analyzeEventsTree()
        inv_mass_jpsi += data[0]
        inv_mass_psi2s += data[1]
        n_jpsi += data[2]
        n_psi2s += data[3]
        bar.update(1)

mass_hist = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=5,name='$|pt|$'))
mass_hist.fill(inv_mass_jpsi)
Plot(mass_hist, title="Invariant mass J/Psi", save=True)
mass_hist2 = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=5,name='$|pt|$'))
mass_hist2.fill(inv_mass_psi2s)
Plot(mass_hist2, title="Invariant mass Psi2S", save=True)
mass_hist_tot = mass_hist + mass_hist2
Plot(mass_hist_tot, title="Invariant mass Monte-Carlo", save=True)
print(f"{int(n_jpsi)} J/psi générés, {int(mass_hist.sum())} détectés, AE = {mass_hist.sum()/n_jpsi}")
print(f"{int(n_psi2s)} Psi(2S) générés, {int(mass_hist2.sum())} détectés, AE = {mass_hist2.sum()/n_psi2s}")


Attends, ça calcul là... :   0%|                              | 0/166 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
pt = ROOT.RooRealVar("pt", "pt", 1.5, 5)
mean1 = ROOT.RooRealVar("mean", "mean of gaussian", 3.1, 1.5, 5)
sigma1 = ROOT.RooRealVar("sigma", "width of gaussian", 0.05, 0, 1)
alpha1 = ROOT.RooRealVar("alpha", "alpha", 1, 0, 5)
n1 = ROOT.RooRealVar("n", "n", 2, 1, 10)
gauss = ROOT.RooCBShape("gauss", "gaussian PDF", pt, mean1, sigma1, alpha1, n1)

pt.setBins(100)  # Set bins for plotting

# Create a RooDataSet and fill it with the data
data_set = ROOT.RooDataSet.from_numpy({"pt": np.array(inv_mass)}, [pt])
# data_set = ROOT.RooDataSet("data_set", "data_set", ROOT.RooArgSet(pt1))
# for value in invariant_mass:
    # pt1.setVal(value)
    # data_set.add(ROOT.RooArgSet(pt1))

xframe = pt.frame(Title="Gaussian pdf with data")  # RooPlot
data_set.plotOn(xframe)
gauss.plotOn(xframe)

gauss.fitTo(data_set)
c = ROOT.TCanvas("rf101_basics", "rf101_basics", 800, 400)
xframe.Draw()


c.SaveAs("rf101_basics.png")